In [9]:
import pandas as pd,geopandas as gpd,numpy as np

In [24]:
cities = pd.read_csv('US_Reg.csv')
cities = cities[cities['Flag.CBSA.availability']==1][['CBSA','CBSA.Name']]
cities = pd.merge(cities,pd.read_csv("US_RegYr.csv")[['CBSA','pop.2010','gdp.2015','emp.2015','pat.2000','pub.2010']].drop_duplicates(),how='left')
cities['CBSA'] = cities['CBSA'].astype(str)

patch = [('31080','31100'),
         ('29200','29140'),
         ('42200','42060'),
         ('48260','44600'),
         ('14010','14060')]
for new,old in patch:
    cities.loc[cities['CBSA']==old,'CBSA']=new
 
msa = gpd.read_file('SHP/cb_2016_us_cbsa_500k.shp')
msa = pd.merge(cities.rename(columns={'CBSA':"CBSAFP"}),msa,how='left')
for col in ['pop.2010','gdp.2015','emp.2015','pat.2000','pub.2010']:
    msa[col] = msa.fillna(0)[col]
    msa['log.'+col] = np.log10(msa.fillna(0)[col])
msa = msa[['CBSAFP','CBSA.Name','pop.2010','gdp.2015','emp.2015','pat.2000','pub.2010','log.pop.2010','log.gdp.2015','log.emp.2015','log.pat.2000','log.pub.2010','geometry']].rename(columns={'CBSAFP':'CBSA'})
msa = gpd.GeoDataFrame(msa.drop('geometry',1),geometry=msa['geometry'])

In [25]:
cones = gpd.GeoDataFrame(msa.drop('geometry',1),geometry=[g.centroid for g in msa['geometry']])

top = 7
for col in ['pop.2010','gdp.2015','emp.2015','pat.2000','pub.2010']:
    cones['Label.'+col.split('.')[0]] = cones['CBSA.Name']
    keep = set(cones.sort_values(by=col,ascending=False)['CBSA'].values.tolist()[:top])
    cones.loc[~cones['CBSA'].isin(keep),'Label.'+col.split('.')[0]] = ''

cones = gpd.GeoDataFrame(cones.drop('geometry',1),geometry=cones['geometry'].values.tolist())

In [26]:
max_height = 1500000
for col in ['pop.2010','gdp.2015','emp.2015','pat.2000','pub.2010']:
    col_max = cones[col].max()
    cones[col.split('.')[0]+'.height'] = (max_height*cones[col]/col_max).round()
    cones[col.split('.')[0]+'.label.height'] = (max_height*cones[col]/col_max).round()

In [27]:
msa.to_file('MSA.shp')
cones.to_file('cones.shp')

In [187]:
def prettyfiNum(n, base = 10,mode='center'):
    if base > 1:
        for i in range(1,10):
            if (n<=base**i)&(n>base**(i-1)):
                break
    else:
        raise NameError('Not supported')
    r = np.linspace(base**(i-1),base**i,base**2-base+1)
    if mode=='min':
        r = r[r<=n]
    elif mode=='max':
        r = r[r>=n]
    diff = r-n
    return r[np.argmin((abs(diff)))]

def prettyfiList(x):
    out = [prettyfiNum(x[0],mode='min')]
    for i in x[1:-1]:
        out.append(prettyfiNum(i))
    out.append(prettyfiNum(x[-1],mode='max'))
    return out

n_cones = 7
lat = 35
lonStart = -65
deltaLon = 1

from shapely.geometry import Point
for col in ['gdp.2015','emp.2015','pat.2000','pub.2010']:
    col_max = cones[col].max()
    geo = [Point(lonStart,lat+i*deltaLon) for i in range(n_cones)]
    legendValues = np.linspace(cones[col].min(),cones[col].max(),n_cones).tolist()
    legendValues = prettyfiList(legendValues)
    legendHeight = [round(max_height*val/col_max) for val in legendValues]
    legendValues = [str(int(round(val))) for val in legendValues]
    legend = gpd.GeoDataFrame(zip(legendValues,legendHeight),columns=['Label','height'],geometry=geo)
    legend.to_file('legend/'+col+'_legend.shp')